In [ ]:
import re
import json
import torch
from transformers.models.qwen2.modeling_qwen2 import Qwen2ForCausalLM
from transformers.models.qwen2.tokenization_qwen2_fast import Qwen2TokenizerFast
from transformers import BitsAndBytesConfig

In [ ]:
MODEL_ID = "Qwen/Qwen2.5-7B-Instruct"

print(f"載入模型: {MODEL_ID} ...")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

try:
    tokenizer = Qwen2TokenizerFast.from_pretrained(MODEL_ID)
    model = Qwen2ForCausalLM.from_pretrained(
        MODEL_ID,
        device_map="auto",
        quantization_config=bnb_config,
        trust_remote_code=True
    )
    print("模型成功載入")
    
    # 測試一段話
    input_text = "你好，請問你能聽到我說話嗎？"
    inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=20)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

except Exception as e:
    print(f"錯誤: {e}")

In [ ]:

def analyze_sentiment_llm(text):
    system_prompt = """
        # Role
        你是一位精通中文語意學與心理學的情感分析專家。你的任務是分析使用者提供的評論文本，並根據下方的「情緒類別學術定義」進行分類。

        # Objective
        1. 閱讀輸入的中文評論。
        2. 分析文本中的顯性情緒詞（Explicit Sentiment）與隱性情緒（Implicit Sentiment，如諷刺、語氣助詞）。
        3. 判斷該評論主要包含的一種或多種情緒。
        4. 嚴格按照定義輸出結果，不允許模糊地帶。

        # 情緒類別與學術定義
        請依據以下心理學與語言學定義進行判斷：

        1. **滿意**
        - **定義**：當個體評估當前情境與其目標一致，或需求得到滿足時產生的正向情緒。
        - **特徵**：包含滿意、興奮、愉悅。在評論中通常表現為對產品/服務的讚賞、推薦意願或感謝。

        2. **憤怒/不滿**
        - **定義**：當個體感知到目標受阻、受到不公待遇或期待落空時產生的激越情緒。
        - **特徵**：包含生氣、敵意、挫折感。在評論中常表現為指責、強烈的負面語氣助詞（如「爛透了」、「氣死」）、或攻擊性語言。

        3. **(悲傷/失望**
        - **定義**：當個體經歷不可挽回的損失（Loss）或無助感（Helplessness）時產生的低落情緒。
        - **特徵**：包含失落、沮喪、遺憾。在評論中表現為對結果的無力感、後悔購買、或委婉的抱怨（如「可惜了...」、「原本以為...」）。

        4. **恐懼/焦慮**
        - **定義**：當個體感知到潛在威脅、不確定性或無法掌控的情況時產生的防禦性情緒。
        - **特徵**：包含擔心、緊張、恐慌。在評論中表現為對產品安全性的疑慮、對售後服務的不確定感、或擔憂。

        5. **厭惡/反感**
        - **定義**：當個體接觸到被認為是有害、不潔、或違反社會規範的事物時產生的排斥反應。
        - **特徵**：包含噁心、鄙視、迴避。在評論中表現為對衛生狀況的批評、對服務態度的極度不屑、或生理上的不適感描述。

        6. **驚訝**
        - **定義**：當個體遭遇非預期（Unexpected）的事件時產生的短暫情緒狀態（可為正向或負向）。
        - **特徵**：包含震驚、錯愕。在評論中表現為「沒想到」、「竟然」、「出乎意料」。
        *注意：若驚訝後緊接著強烈的喜悅或憤怒，請同時標註。*

        7. **中性**
        - **定義**：缺乏明顯的情緒色彩，主要為客觀陳述事實或詢問資訊。
        - **特徵**：不帶個人主觀評價的描述。

        # Rules
        1. 若文本包含諷刺（Sarcasm），請依據其「真實意圖」而非字面意義進行分類（通常歸類為 Disgust 或 Anger）。
        2. 若文本包含多種情緒（Mixed Emotions），請列出所有主要情緒。
        3. 針對中文特有的語氣助詞（如：喔、吧、耶、蛤）需納入情緒強度的判斷依據。

        # Output Format
        請以 JSON 格式輸出，重點在於捕捉所有顯著的情緒：

        {
        "簡短分析": "簡短分析文本中的關鍵詞、語氣轉折與針對的對象...",
        "情緒": [
            {
            "情緒類別": "情緒類別 (使用中文)",
            "情緒強度": 1-5 (強度，5為最強烈),
            "該情緒針對的對象": "該情緒針對的對象 (例如: 服務, 產品, 物流)"
            }
        ]
    """

    user_prompt = f"評論內容：{text}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # 格式化輸入
    text_input = tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )
    
    model_inputs = tokenizer([text_input], return_tensors="pt").to(model.device)

    # 生成回應
    with torch.no_grad():
        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=512,
            temperature=0.1,     # 降低隨機性，讓輸出格式穩定
            top_p=0.9,
            repetition_penalty=1.05 # 避免模型卡死在某個符號
        )

    # 取得生成的片段
    input_len = model_inputs.input_ids.shape[1]
    response = tokenizer.decode(generated_ids[0][input_len:], skip_special_tokens=True)
    
    try:
        # 使用正則表達式尋找第一個 { 和最後一個 } 之間的內容
        match = re.search(r'\{.*\}', response, re.DOTALL)
        if match:
            clean_json = match.group(0)
            return json.loads(clean_json)
        else:
            return {"error": "No JSON found", "raw": response}
    except Exception as e:
        return {"error": str(e), "raw": response}

In [ ]:
# 測試
test_text_list = [
    "網路上有一家500元的吃到飽，大家的評論都很不錯，有各式各樣的菜色，但我實際去了一趟，發現全是便宜的澱粉類，肉片薄得像紙一樣，補菜還慢得要死。",
    "網路上有一家500元的吃到飽，大家的評論都很不錯，有各式各樣的菜色，今天去試了確實CP值很高，雖然海鮮不算頂級，但以500元這個價位來說，該有的都有，跟評論描述的差不多，沒讓人失望，是個粗飽的好選擇。",
    "網路上有一家500元的吃到飽，大家的評論都很不錯，有各式各樣的菜色，結果現場竟然有現切和牛，龍蝦，帝王蟹連甜點都是哈根達斯！",
    "網路上有一家500元的吃到飽，大家的評論都很不錯，有各式各樣的菜色，但是我只看得到炒飯和薯條，而且蒼蠅還幫忙加菜，這種『五星級』體驗真是讓我大開眼界，謝謝大家推薦喔。"
]

for text in test_text_list:
    result = analyze_sentiment_llm(text)
    print(f"輸入評論: {text}\n解析後的結果: {result}\n\n")